In [55]:
import csv
from gensim import corpora, models, similarities
import os

In [56]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [57]:
#load the data
f = open('/Users/mayli/Desktop/ap.csv')
csv_f = csv.reader(f)

#as a list of strings, one per document
documents = []
for row in csv_f:
    documents.append(str(row[1]))

In [58]:
#load stopwords 
f1 = open('/Users/mayli/Desktop/stopwords-es.csv')
csv_f1 = csv.reader(f1)

stopwords = []
for row in csv_f1:
	stopwords.append(row[0])

stoplist = set(stopwords)

In [59]:
# remove common words and tokenize
texts = [[word for word in document.lower().split() if word not in stoplist]
          for document in documents]

In [60]:
# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1] for text in texts]

In [61]:
# build the dictionary, a dict of (question, answer) pair and save it
dictionary = corpora.Dictionary(texts)

2018-09-24 15:31:04,530 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-09-24 15:31:05,523 : INFO : built Dictionary(33657 unique tokens: ['.22', '12th', '16-year-old', '24,', '31,']...) from 2250 documents (total 872980 corpus positions)


In [62]:
# transform the whole texts to vector(sparse) and save it
corpus = [dictionary.doc2bow(text) for text in texts]

In [63]:
#transform to Tf-Idf corpus
tfidf = models.TfidfModel(corpus) 
corpus_tfidf = tfidf[corpus]

2018-09-24 15:31:09,982 : INFO : collecting document frequencies
2018-09-24 15:31:09,987 : INFO : PROGRESS: processing document #0
2018-09-24 15:31:10,247 : INFO : calculating IDF weights for 2250 documents and 33656 features (484311 matrix non-zeros)


In [109]:
#LSI Topics Modeling 
lsi = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=20)
corpus_lsi = lsi[corpus_tfidf]
lsi.print_topics()

for doc in corpus_lsi:
	print(doc)

2018-09-24 16:28:49,044 : INFO : using serial LSI version on this node
2018-09-24 16:28:49,048 : INFO : updating model with new documents
2018-09-24 16:28:51,290 : INFO : preparing a new chunk of documents
2018-09-24 16:28:51,410 : INFO : using 100 extra samples and 2 power iterations
2018-09-24 16:28:51,412 : INFO : 1st phase: constructing (33657, 120) action matrix
2018-09-24 16:28:51,503 : INFO : orthonormalizing (33657, 120) action matrix
2018-09-24 16:28:52,147 : INFO : 2nd phase: running dense svd on (120, 2250) matrix
2018-09-24 16:28:52,191 : INFO : computing the final decomposition
2018-09-24 16:28:52,193 : INFO : keeping 20 factors (discarding 55.891% of energy spectrum)
2018-09-24 16:28:52,242 : INFO : processed documents up to #2250
2018-09-24 16:28:52,247 : INFO : topic #0(6.521): 0.102*"he" + 0.098*"soviet" + 0.098*"his" + 0.096*"bush" + 0.089*"percent" + 0.080*"she" + 0.076*"would" + 0.075*"u.s." + 0.075*"police" + 0.074*"will"
2018-09-24 16:28:52,255 : INFO : topic #1(4

[(0, 0.14615162893179762), (1, -0.061444640114054976), (2, 0.012744949602286346), (3, 0.14532938039551171), (4, -0.0045147225468449396), (5, 0.015155037225649324), (6, 0.022947257372168923), (7, -0.048675833090171143), (8, 0.03933735497000667), (9, 0.00020658769011978095), (10, 0.0043755209074553139), (11, -0.00055533923355351536), (12, -0.056697563994507878), (13, -0.016881549056268687), (14, 0.0079976781336681263), (15, 0.10867622477848231), (16, -0.030552510858992174), (17, 0.024287591039278227), (18, 0.010339326261000663), (19, -0.11358733256522173)]
[(0, 0.14157375505504777), (1, -0.013968617358541553), (2, -0.010405305279006877), (3, -0.015558063866507637), (4, -0.046531555980153926), (5, -0.018721402484566101), (6, -0.0107965805885284), (7, -0.0021157167759424892), (8, -0.077961726320014954), (9, -0.022313934297708952), (10, -0.0010262000371880234), (11, 0.017205772577117414), (12, -0.051183879192227916), (13, -0.032309581016911275), (14, 0.084790446501030611), (15, -0.059733655

[(0, 0.11062976695018813), (1, -0.0076068441985332105), (2, 0.012773183436366136), (3, 0.0012028480512562083), (4, 0.0015400472071454964), (5, -0.0069238430827013066), (6, -0.020524094694699287), (7, 0.006407499258882822), (8, 0.021418036162777179), (9, 0.013377945563183181), (10, 0.023913066543609165), (11, -0.013941807101235951), (12, -0.0021609600292242549), (13, -0.024044212954841118), (14, 0.035567540806055918), (15, -0.05250375943643476), (16, -0.018514297761407852), (17, -0.055616960840188717), (18, 0.010856561880256259), (19, -0.0252500805617102)]
[(0, 0.21554676033983236), (1, -0.043788496750628667), (2, 0.011236763277856313), (3, -0.12579268638580898), (4, 0.057687343499925242), (5, -0.004044361282528745), (6, -0.056479690660021717), (7, 0.029833621034057799), (8, -0.056643901207644752), (9, 0.034579346306144877), (10, 0.033227436397599466), (11, -0.058828016062604502), (12, 0.039936995685824817), (13, -0.0088909205585808275), (14, -0.13506025868162877), (15, 0.09217853509307

[(0, 0.139930732984134), (1, -0.036120043207705024), (2, 0.013993742800510554), (3, 0.059957929257236579), (4, 0.026413025184219074), (5, 0.0055554267079804499), (6, 0.0075902982101055698), (7, -0.03163463000615252), (8, 0.031176217142858068), (9, 0.0042291043826332489), (10, -0.011345238201675619), (11, -0.045948790281002933), (12, -0.0058765311525241894), (13, 0.0044492738801417521), (14, 0.02305989211925627), (15, -0.0040243454753875092), (16, -0.012018390138362459), (17, -0.00096541294161646039), (18, 0.002580582274420692), (19, -0.052694637797148121)]
[(0, 0.13064425304685709), (1, -0.029668301680096879), (2, 0.015082727200474969), (3, -0.11169480043694119), (4, 0.13065343012505684), (5, 0.039582675889008954), (6, 0.043785254458078328), (7, -0.10789126365570376), (8, 0.02960399591441748), (9, -0.024328205477454654), (10, -0.024519842905476368), (11, 0.0036216366277978715), (12, -0.0062392152659578705), (13, -0.044515077346722665), (14, -0.025147169748619171), (15, -0.0193831102952

[(0, 0.10195784253439345), (1, 0.032659902041847384), (2, 0.035816019036701105), (3, -0.010379721169930609), (4, -5.2177863596013398e-05), (5, -0.022612673911752056), (6, -0.071255110749881764), (7, 0.02657353681081619), (8, -0.016821861910751645), (9, 0.025675875313080219), (10, 0.013166488157372539), (11, -0.049089835735650053), (12, 0.017663158235503743), (13, -0.028444782221296438), (14, -0.0049262281811026064), (15, -0.093925690416773525), (16, -0.06150372924740579), (17, -0.037903095385633981), (18, 0.038799752302690324), (19, -0.042777730369731308)]
[(0, 0.095582071831375306), (1, 0.020797474347894935), (2, 0.008378373125150447), (3, -0.010348124722919134), (4, 0.01605963400066239), (5, -0.032731995683129761), (6, -0.074893462047906104), (7, 0.030595198822993146), (8, -0.022017440285787438), (9, 0.024098727453114092), (10, 0.017139405181766969), (11, -0.031914803153974429), (12, 0.022618780579962132), (13, -0.057414529612640326), (14, 0.024806329487111506), (15, -0.0634790989649

[(0, 0.095459253318370671), (1, 0.024904961114967546), (2, 0.022732591384942165), (3, 0.018851384187495081), (4, -0.021756285270457074), (5, -0.0395610787109828), (6, -0.044324935148191705), (7, -0.039185106584718694), (8, 0.044275373205701345), (9, -8.5269714078100998e-05), (10, -0.038466490186099794), (11, -0.0074486382213837217), (12, 0.055063912292219824), (13, 0.030906045733365521), (14, -0.015752329898899478), (15, -0.049058055178557186), (16, 0.036856080586464328), (17, 0.064139986660486245), (18, -0.016379434312760628), (19, 0.010115014689850621)]
[(0, 0.080631979084480276), (1, 0.0016511953404357818), (2, 0.022704918671053329), (3, -0.00069346106983831854), (4, 0.0017677150702634047), (5, -0.02570699553344849), (6, -0.040461891619867185), (7, -0.00061538957555147837), (8, -0.00065664586793894508), (9, 0.016762169971022669), (10, 0.026890896596718443), (11, -0.044072265091227907), (12, -0.0071847383270662972), (13, -0.024702702547232683), (14, 0.015444926809767678), (15, -0.081

In [107]:
#topic distribution for each document
doc_topics = corpus_lsi  # 所有文档的主题分布

dict_doc = {}
docid = 0

for doc in doc_topics:
    
    doc_top = np.array(doc)
    dict_eachdoc = {}
    for i in range(len(doc_top)):
        dict_eachdoc[doc_top[i,0]] = doc_top[i,1]
    print(dict_eachdoc)
    try:
        maxtopic = int(max(dict_eachdoc, key=dict_eachdoc.get))
        print(maxtopic)
        dict_doc[docid] = maxtopic
        docid = docid +1
    except:
        docid = docid +1
#find out the topic with highest lsi value for each doc
    

for key,value in dict_doc.items():
    print('{key}:{value}'.format(key = key, value = value))


{0.0: 0.14610914355854435, 1.0: -0.061479055753236415, 2.0: -0.012932400112410862, 3.0: 0.14484839298218893, 4.0: 0.0073302421553632429, 5.0: -0.014321869562383037, 6.0: -0.023834398891414119, 7.0: -0.048238559060432909, 8.0: -0.038945638973517202, 9.0: 0.0047374372387594841, 10.0: 0.014830164432304907, 11.0: 0.0051580357664349147, 12.0: -0.063049821806487508, 13.0: 0.038092877685855328, 14.0: -0.0096769792763462589, 15.0: -0.096369445198191739, 16.0: 0.053850271985058158, 17.0: 0.044420022238414292, 18.0: 0.0029868254836712647, 19.0: 0.12263054293852972}
0
{0.0: 0.14155546740176686, 1.0: -0.014109361958881413, 2.0: 0.009815276591234906, 3.0: -0.016736568914181762, 4.0: 0.044433575934406354, 5.0: 0.020108944828540056, 6.0: 0.011434025850924883, 7.0: -6.7118188824275657e-06, 8.0: 0.080041537083524711, 9.0: -0.014566784503767685, 10.0: -0.014942576038706589, 11.0: -0.018374800422040757, 12.0: -0.068008962899065564, 13.0: 0.0290635699601417, 14.0: -0.077600978459461692, 15.0: 0.0648214133

0
{0.0: 0.18460926523533602, 1.0: -0.06325338105078121, 2.0: -0.020041007126747185, 3.0: -0.052899730975651468, 4.0: -0.16219884598384601, 5.0: -0.034884906285139898, 6.0: -0.062107926639560253, 7.0: -0.071933197003816049, 8.0: -0.028763995703344713, 9.0: -0.0061628216294792463, 10.0: 5.7418581943978518e-05, 11.0: 0.01091534262053106, 12.0: -0.012552518320796711, 13.0: 0.0070072204251066207, 14.0: -0.021992833728386001, 15.0: 0.017456606207306179, 16.0: -0.015304784931983626, 17.0: -0.0094150618867739153, 18.0: 0.0020732814793807818, 19.0: -0.016662964205591969}
0
{0.0: 0.11231641868249222, 1.0: -0.033959119242869526, 2.0: 0.0066955364569296141, 3.0: 0.050542902439278774, 4.0: 0.055605883801743007, 5.0: -0.0092096442681002342, 6.0: 0.0086053207854191859, 7.0: -0.034467434460697242, 8.0: -0.014118003973522105, 9.0: 0.019388497390516294, 10.0: 0.044375886971093112, 11.0: -0.050670364207823899, 12.0: -0.045017046374624954, 13.0: 0.044261555349360167, 14.0: -0.016426317478300569, 15.0: -0.

{0.0: 0.11207810519647485, 1.0: 0.38965272496878789, 2.0: -0.2986810819333196, 3.0: 0.042359531550633413, 4.0: 0.089430006404713727, 5.0: -0.22846631796469327, 6.0: -0.09012327076014584, 7.0: -0.025032285450304097, 8.0: 0.024755558526464806, 9.0: -0.11196403991390369, 10.0: 0.1013839431579221, 11.0: 0.059848596605794044, 12.0: 0.087373409868070823, 13.0: 0.0076261863878640552, 14.0: 0.035881210763727155, 15.0: -0.032224863699994027, 16.0: 0.0086282409163820819, 17.0: 0.0012787310797493056, 18.0: -0.012333842809600731, 19.0: -0.02325835516222444}
1
{0.0: 0.16391740801954244, 1.0: -0.024311679406786704, 2.0: 0.12951839357853873, 3.0: 0.07745831614206046, 4.0: 0.092436439147709479, 5.0: -0.047399045102299808, 6.0: -0.027616280127082889, 7.0: 0.035580751125262162, 8.0: -0.025681867885576099, 9.0: -0.097212611802477006, 10.0: -0.029340181539398012, 11.0: -0.12635551640077311, 12.0: 0.10022666627799905, 13.0: 0.46905737849008661, 14.0: -0.070767664476239736, 15.0: 0.030641528685242232, 16.0:

{0.0: 0.14749031756959208, 1.0: -0.041405731197956472, 2.0: 0.093182045749192005, 3.0: -0.12656359179472385, 4.0: 0.17480802292507477, 5.0: -0.067683808088022657, 6.0: -0.048695640590632423, 7.0: 0.094713523850774722, 8.0: -0.081157556075007839, 9.0: -0.031179313403438567, 10.0: -0.03774606451805098, 11.0: 0.017098764801464165, 12.0: 0.017288547989787972, 13.0: 0.20621690558700129, 14.0: 0.017612771001577557, 15.0: 0.0429596227344968, 16.0: -0.058352277961270127, 17.0: 0.064794824810129506, 18.0: -0.10026640457282564, 19.0: -0.036888791296325794}
13
{0.0: 0.11110620675590707, 1.0: -0.0068881552390994303, 2.0: -0.015715345205075351, 3.0: 0.0065414917931162211, 4.0: -0.006175702351905097, 5.0: 0.019057911834146644, 6.0: 0.021024860331483687, 7.0: 0.014844085067013643, 8.0: -0.0114692990472001, 9.0: 0.014271237941979065, 10.0: -0.011525486127636438, 11.0: 0.027542650054375346, 12.0: 0.011873080880137523, 13.0: 0.00097666966588650273, 14.0: -0.029969391187859594, 15.0: 0.030250753962418216

{0.0: 0.19928053238852586, 1.0: 0.12134978993214864, 2.0: -0.010557717349213945, 3.0: -0.075849638108984319, 4.0: -0.046995111934032156, 5.0: 0.081255279414416209, 6.0: 0.078798894734236222, 7.0: 0.029948512185832065, 8.0: -0.020422086097816244, 9.0: -0.044583465771731362, 10.0: -0.019404127715443475, 11.0: -0.053385177725613131, 12.0: -0.046288379687337962, 13.0: -0.0056043390416732099, 14.0: 0.020653883317389272, 15.0: -0.067845717729805133, 16.0: -0.007801468887146475, 17.0: -0.0084691223651312611, 18.0: 0.049348974526783648, 19.0: 0.009322108687807586}
0
{0.0: 0.092126464171288974, 1.0: -0.038043545863235086, 2.0: -0.010282832591196944, 3.0: -0.015826509405685908, 4.0: -0.070110462097898923, 5.0: -0.025373946984985758, 6.0: -0.043347408901093176, 7.0: -0.051917936464188297, 8.0: -0.019049308542606159, 9.0: -0.0001094066659521818, 10.0: -0.0030133729381512158, 11.0: 0.013542437499799173, 12.0: -0.01939694704348063, 13.0: 0.0064918815297912149, 14.0: -0.02581225916793203, 15.0: -0.00

267:0
268:1
269:0
270:0
271:0
272:0
273:12
274:0
275:0
276:16
277:0
278:0
279:0
280:3
281:0
282:3
283:0
284:0
285:0
286:0
287:0
288:0
289:0
290:0
291:0
292:1
293:0
294:0
295:0
296:0
297:0
298:11
299:1
300:0
301:0
302:0
303:0
304:1
305:0
306:0
307:0
308:0
309:0
310:0
311:0
312:0
313:0
314:0
315:0
316:0
317:12
318:0
319:0
320:0
321:0
322:0
323:0
324:0
325:10
326:0
327:0
328:0
329:0
330:8
331:12
332:0
333:0
334:0
335:0
336:0
337:0
338:4
339:0
340:0
341:7
342:0
343:0
344:0
345:0
346:0
347:0
348:0
349:0
350:0
351:0
352:0
353:0
354:0
355:0
356:0
357:3
358:3
359:0
360:0
361:0
362:15
363:0
364:0
365:0
366:0
367:0
368:0
369:1
370:0
371:0
372:18
373:7
374:0
375:0
376:0
377:0
378:0
379:0
380:8
381:0
382:13
383:0
384:0
385:0
386:0
387:0
388:0
389:0
390:0
391:0
392:0
393:0
394:0
395:0
396:10
397:0
398:0
399:0
400:0
401:0
402:0
403:0
404:0
405:0
406:0
407:0
408:0
409:0
410:0
411:0
412:0
413:0
414:0
415:0
416:0
417:0
418:0
419:0
420:0
421:0
422:0
423:0
424:0
425:0
426:0
427:0
428:0
429:7
430:0
431:1


1618:0
1619:0
1620:5
1621:0
1622:0
1623:0
1624:6
1625:0
1626:15
1627:0
1628:1
1629:4
1630:0
1631:0
1632:0
1633:0
1634:0
1635:0
1636:3
1637:0
1638:0
1639:0
1640:0
1641:0
1642:0
1643:0
1644:4
1645:7
1646:0
1647:1
1648:0
1649:0
1650:4
1651:0
1652:0
1653:0
1654:0
1655:0
1656:4
1657:0
1658:0
1659:0
1660:0
1661:0
1662:0
1663:1
1664:18
1665:0
1666:7
1667:0
1668:13
1669:0
1670:0
1671:0
1672:0
1673:11
1674:5
1675:0
1676:0
1677:0
1678:13
1679:0
1680:0
1681:0
1682:0
1683:0
1684:0
1685:0
1686:0
1687:0
1688:0
1689:0
1690:0
1691:0
1692:0
1693:0
1694:0
1695:0
1696:0
1697:0
1698:0
1699:0
1700:0
1701:0
1702:0
1703:19
1704:0
1705:0
1706:0
1707:3
1708:0
1709:0
1710:0
1711:0
1712:8
1713:0
1714:2
1715:0
1716:0
1717:0
1718:0
1719:0
1720:0
1721:0
1722:0
1723:0
1724:0
1725:18
1726:0
1727:0
1728:0
1729:3
1730:0
1731:2
1732:1
1733:0
1734:9
1735:0
1736:0
1737:0
1738:12
1739:0
1740:0
1741:0
1742:0
1743:3
1744:0
1745:0
1746:0
1747:17
1748:0
1749:0
1750:18
1751:0
1752:0
1753:0
1754:0
1755:1
1756:0
1757:0
1758:0
175

In [67]:
#LDA Topics Modeling
import numpy as np
lda = models.LdaModel(corpus_tfidf, id2word=dictionary, num_topics=20, passes = 5)
num_topics = 20
#num_show_term = 7   # 每个主题下显示几个词
for topic_id in range(num_topics):
    print('Topic %d：\t' % topic_id)
    term_distribute = np.array(lda.get_topic_terms(topicid=topic_id))
    term_id = term_distribute[:, 0].astype(np.int)
    print('Words：\t', end='  ')
    for t in term_id:
        print(dictionary.id2token[t], end=' ')
    print('\nPossibility：\t', term_distribute[:, 1])



2018-09-24 15:35:24,812 : INFO : using symmetric alpha at 0.05
2018-09-24 15:35:24,816 : INFO : using symmetric eta at 0.05
2018-09-24 15:35:24,825 : INFO : using serial LDA version on this node
2018-09-24 15:35:24,950 : INFO : running online (multi-pass) LDA training, 20 topics, 5 passes over the supplied corpus of 2250 documents, updating model once every 2000 documents, evaluating perplexity every 2250 documents, iterating 50x with a convergence threshold of 0.001000
2018-09-24 15:35:24,952 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2018-09-24 15:35:26,813 : INFO : PROGRESS: pass 0, at document #2000/2250
2018-09-24 15:35:28,145 : INFO : merging changes from 2000 documents into a model of 2250 documents
2018-09-24 15:35:28,211 : INFO : topic #18 (0.050): 0.001*"mrs." + 0.001*"percent" + 0.001*"she" + 0.001*"her" + 0.001*"police" + 0.001*"he" + 0.001*"his" + 0.001*"students" + 0.000*"were" + 0.0

2018-09-24 15:35:36,568 : INFO : topic #7 (0.050): 0.002*"thompson" + 0.002*"koch" + 0.002*"federated" + 0.002*"pont" + 0.002*"6.3" + 0.001*"du" + 0.001*"mash" + 0.001*"frohnmayer" + 0.001*"allied" + 0.001*"incest"
2018-09-24 15:35:36,570 : INFO : topic #15 (0.050): 0.002*"apple" + 0.002*"bentsen" + 0.002*"n;" + 0.001*"lutheran" + 0.001*"earthquake" + 0.001*"shark" + 0.001*"mecham" + 0.001*"y;" + 0.001*"dukakis" + 0.001*"jackson"
2018-09-24 15:35:36,572 : INFO : topic #9 (0.050): 0.001*"killers" + 0.001*"afghanistan's" + 0.001*"china" + 0.001*"kabul" + 0.001*"hoan" + 0.001*"10-minute" + 0.001*"afghan" + 0.001*"lafayette" + 0.001*"keefe" + 0.001*"milan"
2018-09-24 15:35:36,576 : INFO : topic diff=0.413303, rho=0.492366
2018-09-24 15:35:38,439 : INFO : PROGRESS: pass 3, at document #2000/2250
2018-09-24 15:35:39,095 : INFO : merging changes from 2000 documents into a model of 2250 documents
2018-09-24 15:35:39,158 : INFO : topic #7 (0.050): 0.001*"thompson" + 0.001*"koch" + 0.001*"federa

Topic 0：	
Words：	  cents cent hyundai futures lower champion quarter diet excerpts fiscal 
Possibility：	 [ 0.00319225  0.00276147  0.00198446  0.00193413  0.00174087  0.00134451
  0.00118597  0.00118543  0.00116229  0.00114509]
Topic 1：	
Words：	  cdy clr bendjedid executed users penalty yeutter inches cisneros list 
Possibility：	 [ 0.00169101  0.0015161   0.00141986  0.00137707  0.00118472  0.00102152
  0.00092637  0.00087192  0.00086024  0.00084633]
Topic 2：	
Words：	  wednesday's literary imperial m princess emperor cdc royal royalties petty 
Possibility：	 [ 0.00182806  0.00144293  0.00138813  0.00130676  0.00123556  0.0011663
  0.00113252  0.00108087  0.00102619  0.00087805]
Topic 3：	
Words：	  german germany east maung soybean germany's kohl west unification shuttle 
Possibility：	 [ 0.00279053  0.00243243  0.0019294   0.00160187  0.00151167  0.00137001
  0.00118456  0.00116263  0.00116121  0.00110124]
Topic 4：	
Words：	  iraqi iraq saudi roberts u.n. moslem iran restore iranian arabia

In [68]:
#from pprint import pprint
import numpy as np
corpus_lda = lda[corpus_tfidf]

for doc in corpus_lda:
    print(doc)
    

[(0, 0.025712548), (4, 0.010329716), (6, 0.05639305), (8, 0.18789084), (10, 0.068562359), (13, 0.26747841), (14, 0.013923092), (16, 0.022959782), (17, 0.042944327), (18, 0.2596696)]
[(4, 0.020111127), (6, 0.027499275), (10, 0.13009576), (13, 0.13357644), (17, 0.33587542), (18, 0.28321218)]
[(5, 0.044551414), (6, 0.014605455), (10, 0.058010358), (13, 0.11868862), (14, 0.028560402), (17, 0.03745614), (18, 0.6459617)]
[(0, 0.033677697), (1, 0.01244426), (3, 0.022944618), (4, 0.010478428), (5, 0.20683281), (6, 0.041100841), (10, 0.11992724), (13, 0.25102296), (15, 0.020996051), (16, 0.058957517), (17, 0.055472173), (18, 0.14013419)]
[(6, 0.017819226), (7, 0.026860466), (8, 0.023760539), (9, 0.2725625), (10, 0.11217206), (13, 0.11307219), (15, 0.014600851), (17, 0.035330873), (18, 0.3278977)]
[(4, 0.014793151), (6, 0.036261816), (10, 0.50832856), (13, 0.14445682), (15, 0.031022735), (17, 0.046378389), (18, 0.16660406)]
[(0, 0.027676523), (10, 0.54997194), (13, 0.16203542), (14, 0.041294608)

[(0, 0.023626374), (3, 0.025112426), (6, 0.03918862), (8, 0.1266413), (10, 0.13270612), (13, 0.1907744), (14, 0.063401304), (17, 0.068888865), (18, 0.28922147)]
[(0, 0.029031349), (5, 0.020484053), (6, 0.23856694), (8, 0.019487554), (10, 0.14817792), (13, 0.19856454), (16, 0.021218689), (17, 0.045501873), (18, 0.23643814)]
[(0, 0.01325672), (1, 0.013256719), (2, 0.38085896), (3, 0.013256719), (4, 0.013256719), (5, 0.013256719), (6, 0.013256719), (7, 0.013256719), (8, 0.013256719), (9, 0.013256719), (10, 0.38052011), (11, 0.013256719), (12, 0.013256719), (13, 0.013256719), (14, 0.013256719), (15, 0.013256719), (16, 0.013256721), (17, 0.013256719), (18, 0.013256719), (19, 0.013256719)]
[(5, 0.024396172), (6, 0.037815295), (10, 0.22192723), (13, 0.46902639), (16, 0.017557336), (18, 0.16934995)]
[(3, 0.029207798), (10, 0.056789283), (11, 0.37004668), (13, 0.22871701), (14, 0.032219667), (15, 0.02793858), (16, 0.02728866), (17, 0.024706002), (18, 0.13866416)]
[(1, 0.069884948), (4, 0.313179

[(1, 0.013446141), (4, 0.014057271), (5, 0.012805299), (6, 0.12601726), (8, 0.1984707), (10, 0.12304711), (13, 0.1995524), (14, 0.029783534), (17, 0.060897607), (18, 0.16212927), (19, 0.013743563)]
[(5, 0.017795343), (6, 0.030279253), (10, 0.13261272), (12, 0.017141394), (13, 0.08045733), (16, 0.031576838), (17, 0.019265212), (18, 0.61109298), (19, 0.012934921)]
[(10, 0.1166278), (13, 0.66306728), (17, 0.011358148), (18, 0.13329084)]
[(1, 0.019434603), (7, 0.14192338), (10, 0.20693314), (13, 0.13643694), (15, 0.018497491), (16, 0.024352023), (17, 0.079993367), (18, 0.30655023)]
[(2, 0.20910443), (3, 0.020917729), (4, 0.010658042), (5, 0.020891253), (6, 0.1144339), (10, 0.065396525), (11, 0.022374889), (13, 0.28515995), (17, 0.026316378), (18, 0.17806952)]
[(3, 0.021047518), (4, 0.024740143), (6, 0.020559035), (10, 0.088905111), (13, 0.20790558), (17, 0.014118697), (18, 0.56504959)]
[(3, 0.038065694), (4, 0.10457645), (6, 0.011539655), (9, 0.21451987), (10, 0.21610594), (13, 0.074906155

[(0, 0.01609076), (3, 0.020614166), (4, 0.018727336), (6, 0.034837555), (10, 0.22541611), (13, 0.23716675), (16, 0.24008034), (18, 0.16418138)]
[(2, 0.014067966), (3, 0.29706243), (4, 0.017872581), (6, 0.032429621), (8, 0.010206324), (10, 0.069334686), (13, 0.074261732), (14, 0.072697833), (15, 0.014415937), (16, 0.058395859), (17, 0.019042259), (18, 0.28633627)]
[(0, 0.01497467), (3, 0.013015605), (5, 0.044017706), (6, 0.067292579), (10, 0.18370374), (11, 0.016677765), (12, 0.19614041), (13, 0.2121433), (14, 0.012357858), (15, 0.030122504), (16, 0.035877049), (17, 0.053625975), (18, 0.093196593)]
[(0, 0.012872894), (1, 0.012746956), (3, 0.08992321), (6, 0.022189405), (10, 0.099869594), (12, 0.17595069), (13, 0.25835663), (16, 0.024754276), (17, 0.045754101), (18, 0.22011846)]
[(0, 0.023140589), (2, 0.015885588), (5, 0.013989994), (7, 0.017409576), (10, 0.15523489), (12, 0.046685044), (13, 0.21012883), (14, 0.29849562), (16, 0.034025867), (17, 0.032397147), (18, 0.11315141)]
[(1, 0.015

[(3, 0.020704385), (4, 0.011097113), (6, 0.022802249), (8, 0.19772713), (10, 0.2473858), (13, 0.1951039), (14, 0.031633478), (16, 0.013199119), (17, 0.055287652), (18, 0.15319447)]
[(0, 0.0186528), (3, 0.19214477), (4, 0.013262685), (6, 0.074634306), (8, 0.016740749), (10, 0.12830454), (12, 0.013274063), (13, 0.2153739), (15, 0.010368188), (17, 0.044886682), (18, 0.23351534)]
[(0, 0.017554561), (4, 0.022220938), (10, 0.70422119), (13, 0.067206301), (14, 0.023144258), (17, 0.031193957), (18, 0.055762354)]
[(0, 0.026914394), (1, 0.11004794), (3, 0.047711998), (8, 0.012513186), (10, 0.20506309), (13, 0.10277771), (14, 0.02751936), (16, 0.014533128), (17, 0.05555483), (18, 0.33250439), (19, 0.020499162)]
[(2, 0.14884312), (4, 0.022413986), (6, 0.087260775), (10, 0.080322929), (13, 0.3514455), (17, 0.07733617), (18, 0.17692918)]
[(0, 0.017525816), (10, 0.024303649), (13, 0.71120131), (17, 0.014766201), (18, 0.13175449), (19, 0.015970722)]
[(4, 0.01666774), (6, 0.40022159), (10, 0.11228172),

...

[(0, 0.018011777), (3, 0.018123141), (4, 0.013219962), (6, 0.032389693), (9, 0.18438415), (10, 0.21198562), (13, 0.12422586), (14, 0.04992494), (15, 0.014542758), (17, 0.037614278), (18, 0.26044661)]
[(1, 0.027383212), (4, 0.42147243), (5, 0.016192215), (10, 0.087123923), (15, 0.023161447), (17, 0.094942436), (18, 0.25852343)]
[(3, 0.028070901), (4, 0.020616913), (6, 0.046800848), (7, 0.076525204), (10, 0.086355232), (12, 0.04889581), (13, 0.09846887), (14, 0.40094224), (17, 0.031017253), (18, 0.12162846)]
[(5, 0.052369453), (10, 0.10061349), (11, 0.023463452), (13, 0.63767511), (17, 0.018512651), (18, 0.085808903)]
[(0, 0.13051693), (3, 0.011940979), (4, 0.025875781), (6, 0.033524763), (8, 0.021603901), (9, 0.017140228), (10, 0.1544634), (13, 0.28488165), (16, 0.010913688), (17, 0.033133551), (18, 0.24842803)]
[(2, 0.15733968), (3, 0.066095054), (10, 0.11244866), (11, 0.035771932), (13, 0.21061346), (14, 0.038518745), (16, 0.041467339), (17, 0.084377475), (18, 0.20379496)]
[(3, 0.0184

[(5, 0.023653733), (10, 0.040139865), (13, 0.086221978), (15, 0.010212312), (18, 0.76953995)]
[(3, 0.010995278), (6, 0.022643829), (10, 0.16227868), (13, 0.52711087), (14, 0.013410982), (17, 0.058401961), (18, 0.15746537)]
[(3, 0.034948952), (6, 0.10055654), (8, 0.24510233), (13, 0.35286641), (16, 0.018252799), (17, 0.024888208), (18, 0.15227753)]
[(2, 0.010326431), (3, 0.018791463), (4, 0.056441993), (6, 0.054476064), (10, 0.25677696), (13, 0.19163671), (14, 0.042068284), (17, 0.037386708), (18, 0.17762011), (19, 0.099540681)]
[(0, 0.028248424), (4, 0.010329544), (7, 0.015015502), (9, 0.20794548), (10, 0.089641809), (13, 0.27791056), (14, 0.037080217), (15, 0.018124366), (17, 0.046306375), (18, 0.21071653)]
[(2, 0.011286951), (4, 0.28206593), (6, 0.04968033), (10, 0.10771479), (13, 0.067755163), (14, 0.089558102), (15, 0.057769071), (16, 0.03892222), (17, 0.030675432), (18, 0.22036715)]
[(0, 0.10962909), (9, 0.24099843), (10, 0.052923415), (13, 0.032565154), (16, 0.033137284), (17, 0.

[(3, 0.022186566), (4, 0.014725168), (6, 0.033978928), (10, 0.16560462), (13, 0.23968124), (14, 0.019876733), (15, 0.017766424), (16, 0.024743283), (17, 0.037801985), (18, 0.21844228), (19, 0.17912687)]
[(3, 0.067812458), (10, 0.0364778), (13, 0.13798057), (18, 0.66536337)]
[(3, 0.028170556), (6, 0.070032924), (10, 0.37380609), (11, 0.021498706), (13, 0.19338366), (14, 0.037013337), (17, 0.043578401), (18, 0.19075847)]
[(3, 0.037182145), (6, 0.011021496), (7, 0.057411741), (10, 0.094003946), (13, 0.27118737), (14, 0.26022148), (17, 0.026690237), (18, 0.18490057)]
[(10, 0.11981048), (13, 0.014145629), (14, 0.014540017), (17, 0.02462046), (18, 0.73991787)]
[(0, 0.013070194), (4, 0.034690406), (6, 0.015052864), (8, 0.035080735), (10, 0.35570976), (12, 0.090002611), (13, 0.11012162), (14, 0.054413866), (16, 0.019962359), (17, 0.072497159), (18, 0.15556479)]
[(1, 0.19176333), (3, 0.012565257), (4, 0.011966109), (8, 0.039318483), (9, 0.011428827), (10, 0.11751226), (13, 0.15973254), (14, 0.0

[(3, 0.024603104), (5, 0.026914893), (6, 0.059988916), (8, 0.013297279), (10, 0.037893493), (13, 0.55748689), (17, 0.0220343), (18, 0.21173508)]
[(1, 0.04268682), (3, 0.024730692), (6, 0.047872759), (8, 0.026281614), (13, 0.61318284), (16, 0.054666016), (18, 0.10914288)]
[(1, 0.043064978), (4, 0.025417056), (5, 0.098672718), (10, 0.029298471), (14, 0.053761981), (18, 0.66646141)]
[(1, 0.088841565), (3, 0.05860398), (4, 0.084560052), (8, 0.061168078), (10, 0.016034279), (11, 0.01160495), (13, 0.23662527), (15, 0.044866662), (17, 0.011587781), (18, 0.31549194), (19, 0.032957152)]
[(0, 0.047765661), (3, 0.099674515), (6, 0.017678024), (9, 0.022046158), (13, 0.36877188), (16, 0.12314295), (18, 0.21945207), (19, 0.031964779)]
[(1, 0.017271919), (2, 0.026415257), (4, 0.015743844), (6, 0.051400047), (9, 0.01654252), (10, 0.36555806), (12, 0.017273078), (16, 0.072157532), (17, 0.2513859), (18, 0.11999062)]
[(4, 0.028532499), (5, 0.026434811), (6, 0.093150184), (7, 0.030861219), (13, 0.23571248

[(0, 0.055399522), (1, 0.029178372), (3, 0.14176694), (4, 0.039222464), (6, 0.083045878), (10, 0.16017869), (11, 0.016829984), (13, 0.087593742), (16, 0.06672544), (18, 0.27858421)]
[(0, 0.038004555), (4, 0.046975914), (6, 0.31353542), (8, 0.056286834), (14, 0.028523542), (17, 0.13723512), (18, 0.27141809), (19, 0.042782553)]
[(0, 0.20553486), (3, 0.16472268), (5, 0.16577813), (12, 0.024685154), (15, 0.014935186), (16, 0.01575534), (18, 0.34157345)]
[(2, 0.02467159), (3, 0.042199381), (5, 0.015662765), (6, 0.20049445), (8, 0.042284071), (9, 0.041360617), (10, 0.028635021), (12, 0.031868197), (13, 0.17332298), (14, 0.06010573), (15, 0.011950061), (18, 0.29131213)]
[(2, 0.054287255), (3, 0.13724419), (4, 0.013682764), (7, 0.02334073), (9, 0.013142142), (10, 0.28884482), (11, 0.036828119), (13, 0.13993318), (16, 0.051311169), (18, 0.20025969)]
[(16, 0.75401682), (18, 0.10479439)]
[(0, 0.14407763), (3, 0.055575732), (4, 0.052280094), (5, 0.013948945), (10, 0.079682328), (12, 0.025061384), 

In [102]:
#topic distribution for each document
doc_topics = lda.get_document_topics(corpus_tfidf)  # 所有文档的主题分布

dict_doc = {}
docid = 0

for doc in doc_topics:
    
    doc_top = np.array(doc)
    dict_eachdoc = {}
    for i in range(len(doc_top)):
        dict_eachdoc[doc_top[i,0]] = doc_top[i,1]
    #print(dict_eachdoc)
    maxtopic = int(max(dict_eachdoc, key=dict_eachdoc.get))
    #print(maxtopic)
    
    dict_doc[docid] = maxtopic
    docid = docid +1

for key,value in dict_doc.items():
    print('{key}:{value}'.format(key = key, value = value))



0:13
1:17
2:18
3:13
4:18
5:10
6:10
7:10
8:17
9:10
10:11
11:15
12:13
13:10
14:10
15:19
16:10
17:18
18:18
19:18
20:10
21:14
22:14
23:17
24:18
25:4
26:13
27:18
28:13
29:18
30:10
31:18
32:15
33:10
34:18
35:13
36:13
37:10
38:4
39:0
40:10
41:18
42:18
43:10
44:13
45:13
46:16
47:13
48:18
49:10
50:10
51:14
52:16
53:0
54:13
55:13
56:13
57:7
58:13
59:14
60:6
61:10
62:10
63:18
64:14
65:10
66:18
67:18
68:13
69:13
70:18
71:18
72:16
73:7
74:13
75:18
76:6
77:11
78:4
79:14
80:18
81:14
82:18
83:0
84:3
85:16
86:10
87:18
88:10
89:5
90:17
91:17
92:14
93:17
94:18
95:18
96:10
97:8
98:15
99:7
100:18
101:6
102:2
103:13
104:11
105:4
106:18
107:13
108:0
109:10
110:13
111:13
112:18
113:17
114:18
115:13
116:8
117:18
118:10
119:10
120:19
121:14
122:13
123:13
124:17
125:0
126:13
127:18
128:8
129:18
130:2
131:10
132:18
133:3
134:10
135:5
136:18
137:0
138:4
139:10
140:18
141:18
142:13
143:18
144:18
145:17
146:4
147:18
148:15
149:13
150:11
151:13
152:18
153:18
154:10
155:14
156:10
157:18
158:18
159:18
160:14
161:13
162